In [9]:
import pandas as pd
import numpy as np
import sqlparse
import re
from SQLParser import *

In [10]:
raw = '''
SELECT DISTINCT t.EQ, t.CURR, t.INN, t3.code as REPOSITARY_ID,
CASE WHEN t1.SHORT_NAME is NULL THEN
CASE WHEN t2.SHORT_NAME is NULL THEN 
CASE WHEN t4.SHORT_NAME is NULL THEN 'NOT FOUND'
ELSE t4.SHORT_NAME END 
ELSE t2.SHORT_NAME END 
ELSE t1.SHORT_NAME END as SHORT_NAME,
CASE WHEN t1.NAME is NULL THEN
CASE WHEN t2.NAME is NULL THEN 
CASE WHEN t4.FULL_NAME is NULL THEN 'NOT FOUND'
ELSE t4.FULL_NAME END
ELSE t2.NAME END
ELSE t1.NAME END as NAME, 
CASE WHEN t1.EXCHANGE_ID = 'MICX' THEN 1 
WHEN t1.EXCHANGE_ID = 'PICX' THEN 2
WHEN t1.EXCHANGE_ID = 'NICX' THEN 3
ELSE 9 END as number_ 
FROM uniq_inn t LEFT JOIN zp_dm_vdrf.SV_EQ_FIRMS t1 on t1.FIRM_ID = t.EQ 
LEFT JOIN  zp_dm_vdrf.SV_CURR_FIRMS t2 on t2.FIRM_ID = t.CURR
LEFT JOIN zp_dm_nrd.st_person t3 ON t.INN = t3.INN 
INNER JOIN zp_dm_nrd.st_person_j t4 ON t3.ID = t4.ID 
WHERE t3.INN is not null and t3.inn not like '%-' and not rlike(t3.inn, '^0+') and not rlike(t3.inn, '^1+') and t3.person_type=1 
and not rlike(t3.code,'^\d')
UNION 
SELECT DISTINCT '' as EQ,'' as URR, INN, 
t.code as REPOSITARY_ID, t1.SHORT_NAME as SHORT_NAME, t1.FULL_NAME as NAME, 10 as number_ 
FROM zp_dm_nrd.st_person  t INNER JOIN zp_dm_nrd.st_person_j t1 ON t.id = t1.id 
WHERE t.INN not in (SELECT INN FROM uniq_inn) 
and t.INN is not null and t.inn not like '%-' and not rlike(t.inn, '^0+') and not rlike(t.inn, '^1+') and t.person_type = 1
'''


In [11]:
sql = SQLParser()

In [16]:
var = sql.replace_tabs_from_string(raw)
var = sqlparse.format(var, reindent=True, identifier_case='lower', keyword_case='upper')
var = sql.replace_where_stmnt(var)
assert type(var) == str
var = sql.replace_select_stmnt(var)
var

"SELECT DISTINCT \n \t\t t.eq,\n                t.curr,\n                t.inn,\n                t3.code AS repositary_id,\n                CASE\n                    WHEN t1.short_name IS NULL THEN CASE\n                                                        WHEN t2.short_name IS NULL THEN CASE\n                                                                                            WHEN t4.short_name IS NULL THEN 'NOT FOUND'\n                                                                                            ELSE t4.short_name\n                                                                                        END\n                                                        ELSE t2.short_name\n                                                    END\n                    ELSE t1.short_name\n                END AS short_name,\n                CASE\n                    WHEN t1.name IS NULL THEN CASE\n                                                  WHEN t2.name IS NULL THEN C